# KoBigBird Keyword Extraction

sts 파인 튜인을 한 KoBigBird 모델로 글의 핵심 키워드를 추출하는 모델을 학습니다.

여기서 핵심 키워드란 전체 글의 내용을 대표하는 키워드를 말합니다. 

또한 글의 제목의 내용도 참고합니다. 글의 제목은 글을 한 문장으로 요약한 것이라고 볼 수 있기 때문에 글의 본문 내용뿐만 아니라 글의 제목도 키워드 추출에 사용합니다.

키워드 추출 과정은 다음과 같습니다.

1. 본문 텍스트를 형태소 분석하여 명사/명사구를 추출한다. 이 단어들은 후보 키워드들이 된다.
2. 불용어 사전을 통해 후보 키워드를 정제한다.
3. fine-tuning 한 KoBigBird 모델을 통해 각 후보 키워드들의 표현 벡터를 구한다.
4. fine-tuning 한 KoBigBird 모델을 통해 글의 제목과 본문의 표현도 구한다.
5. 글의 제목 벡터와 본문의 표현을 element weighted sum을 한다.
6. 각 후보 키워드들의 벡터와 글의 제목과 본문의 weighted sum을 한 벡터의 유사도를 계산한다.
7. 가장 유사도가 높은 상위 k의 키워드를 추출한다.

## 본문 키워드 형태소 분석하여 후보 키워드 추출하기

형태소 분석기 komoran을 사용하기 위해 konlpy 설치

## 불용어(stopwords) 사전 정의

두 개의 한국어 불용어 사전을 이용

In [1]:
import csv

stopwords = []
with open('./data/stopwords/stopwords.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))

with open('./data/stopwords/stopwords-ko.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))
        
with open('./data/stopwords/stopwords-np.txt', 'r') as f:
    tsv_file = csv.reader(f, delimiter="\t")
    for line in tsv_file:
        stopwords.append(line[0])

stopwords_set = set(stopwords)
print(stopwords_set)

{'어이', '몇', '하마터면', '여', '경우', '없', '요만한걸', '논하지 않다', '할만하다', '％', '것과 같이', '문제', '각자', '모두', '해서는 안된다', '보', '8', '어째서', '할수있어', '\\', '로', '번', '그리고', '또', '같', '네', '관계가 있다', '삼', '여차', '어느쪽', '에서', '앗', '총적으로 말하면', '저', '하려고하다', '잇따라', '혹시', '$', '않기 위해서', '할지라도', '이어서', '하지마라', '지금', '향해서', '하겠는가', '가령', '중', '그럼', '이용하여', '그들', '다른', '이와 같은', '소리', '과연', '자기집', '의지하여', '되', '참나', '할망정', '9', '따라', '들', '전자', '견지에서', '남짓', '연관되다', '저기', '실로', '으로 인하여', '어느해', '～', '좋', '얼마', '버금', '오자마자', '곧', '？', '저것만큼', '헐떡헐떡', '바로', '5', '일것이다', '｛', '시각', '과', '９', '둘', '놓', '까지 미치다', '하느니', '둥둥', '다소', '데', '의해', '공동으로', '대해서', '여섯', '오직', '인 듯하다', '그러', '그때', '4', '그럼에도 불구하고', '하면된다', '해요', '에 대해', '이 정도의', '하기 위하여', '여보시오', '바꾸어서 한다면', '일지라도', '·', '어찌하여', '매', '하는 김에', '이번', '토하다', '향하다', '그리하여', '더군다나', '좋아', '、', '안', '거니와', '대로 하다', '아니나다를가', '살', '하여금', '뚝뚝', '아니었다면', '끼익', '하면 할수록', '예를 들자면', '그래', '로부터', '옆사람', '까악', '것들', '딩동', '따위', '할 줄 안다', '언제', '하는것만 못하다', '싶', '허', '+', '〉', 

브런치 샘플 데이터 불러오기

In [2]:
import jsonlines

sample = []

with jsonlines.open("./data/brunch/brunch_dataset_sample.jsonl") as f:
    for line in f.iter():
        sample.append(line)

### 불용어 제거 함수

In [3]:
def remove_stopwords(nouns, stopwords):
    for n in nouns:
        if n in stopwords:
            nouns.remove(n)
    return nouns

### 영어 단어 명사 추출 함수

In [4]:
import nltk
from nltk.corpus import stopwords
from string import punctuation

def get_nouns_eng(text):

    text = text.encode('utf-8').decode('ascii', 'ignore')

    result = ""
    for t in text:
        if t not in punctuation:
            result += t

    text = result.strip()

    word_tokens = nltk.word_tokenize(text)
    tokens_pos = nltk.pos_tag(word_tokens)
    NN_words = []
    for word, pos in tokens_pos:
        if 'NN' in pos or 'NNP' in pos:
            NN_words.append(word)
    NN_words = set(NN_words)
    NN_words = list(NN_words)
    return NN_words

In [5]:
get_nouns_eng(sample[1]['body_text'])

['ChatGPT', 'OpenAI', 'prompt', 'GPT', 'SNS', 'imagine', 'AI', 'Midjourney']

### 각 형태소 분석기별 명사 추출 함수

형태소 분석을 통해 명사 추출, 불용어 제거, 영어 단어 추출하는 함수

In [7]:
!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)


In [8]:
from konlpy.tag import Komoran
from konlpy.tag import Kkma
from konlpy.tag import Okt

def get_nouns_komoran(text):
    komoran = Komoran()
    nouns = komoran.nouns(text)
    nouns = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    nouns.extend(eng_nouns)
    
    return nouns

def get_nouns_kkma(text):
    kkma = Kkma()
    nouns = kkma.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    nouns.extend(eng_nouns)
    return nouns

def get_nouns_okt(text):
    okt = Okt()
    nouns = okt.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    nouns.extend(eng_nouns)
    
    return nouns

In [12]:
sample_nouns = get_nouns_komoran(sample[1]['body_text'])
print(sample[1]['title'])
print(sample[1]['body_text']+'\n')
print(sample_nouns)

AI로 이미지 만들기 (매우 쉬움)

요즘 AI로 인터넷이 난리도 아니다. 특히 챗GPT. OpenAI사에서 신규로 론칭한 AI 챗봇 프로그램인데 기세가 심상치 않다. 뼛속까지 문과인 나조차 AI분야에 관심을 갖게 만드는 공상과학과 유사한 엄청난 기술을 자랑한다. 프로토타입인 데다 트래픽이 많아 직접 사용해보진 못했으나 링크드인이나 유튜브를 통해 챗GPT의 능력을 쉽게 접할 수 있다. "언어"로 가능한 모든 영역을 수행할 수 있을뿐더러 코딩까지 대신 써준다. 예를 들어, 천만 관객용 영화의 플롯과 인물 구성을 써달라고 요청하면 몇 분 내로 완벽한 플롯과 인물 구성을 내놓는다. 뉴스 기사, 시나리오, 산문, 시, 소네트, 심지어는 마케팅 전략까지 써준다. 인터넷에 업로드되어 있는 무한에 가까운 정보 풀과 고도화된 머신러닝의 결합물이라고 볼 수 있다. 

어떤 질문에도 답을 내놓아 구글을 대체할 것이라는 목소리도 나온다. 구글은 사용자가 직접 정보를 모아 분석해서 답을 도출해야 하지만 챗GPT는 중간 과정을 생략하고 질문 하나로 바로 해답에 다다를 수 있게 해 준다. 잠재력을 인정받아 마이크로소프트로부터 막대한 투자를 여러 차례 받고 있기도 하다. (1월 24일 기준 1조 원 넘게 투자한 것으로 밝혀졌다.) 해당 프로그램에 대한 정보를 찾다 보면 놀라움의 연속이 아닐 수 없다. 하지만 걷잡을 수 없이 빠르게 진화하는 기술로 인한 불안과 공포도 분명 존재한다. 확실한 건 인류가 수십 년간 그려왔던 미래가 껑충 앞으로 다가와 더 선명해졌다는 것이다. AI의 세계에 몇 시간 정신을 팔다 보면 실제로 미래에 와 있다는 생각이 들 정도다. 

최근 ChatGPT의 인기로 인해 나의 감각도 AI를 향해 뻗어있다. 그래서 그런지, AI관련 기술에 대한 기사나 SNS 포스팅, 유튜브 영상을 많이 찾아보는 요즘이다. (알고리즘에 의해 의도적으로 노출되고 있는 거겠지만.) 그러던 중 며칠 전 한 링크드인 포스팅에서 또 다른 AI툴을 발견했다. 툴의 이름은 바로 Midjourney,